# Ollama Finetuning
* transformers fix: pip install transformers==2.4.1

In [44]:
from llama_index.llms.ollama import Ollama
from llama_index.core.bridge.pydantic import BaseModel
from loguru import logger

## config

In [45]:
class configLLM:
    model_name = "llama3.1:latest"
    request_timeout = 240.0
    json_mode= True
    context_window = 8000

## set LLM

In [46]:
params = configLLM()

llm = Ollama(
    model=params.model_name,
    request_timeout=params.request_timeout,
    context_window=params.context_window,
    json_mode=params.json_mode
)


### structured output

In [47]:
class StructuredOutput(BaseModel):
    """
    Example of structured output
    """
    answer: str
    confidence: float
    explanation: str
    llm_name: str = llm.model

structured_llm = llm.as_structured_llm(StructuredOutput)

In [48]:
resp = structured_llm.complete("Who is Paul Graham?")

In [49]:
import json

str_resp = resp.model_dump_json()

In [50]:
from docx import Document

# convert to dict
dict_resp = json.loads(str_resp)
logger.info(f"Response: {dict_resp}")

2025-07-08 17:45:28.662 | INFO     | __main__:<module>:5 - Response: {'text': '{"answer":"Paul Graham is a well-known British-American programmer, writer, and venture capitalist. He\'s best known for co-founding the online startup incubator Y Combinator in 2005, which has invested in many successful companies like Dropbox, Airbnb, and Reddit.","confidence":1.0,"explanation":"Paul Graham was born in London in 1964 but moved to the US in his youth. He studied at Cambridge University, then Harvard, where he co-founded Viaweb (later renamed as Yahoo! Store) with Robert Tappan Morris.","llm_name":"llama3.1:latest"}', 'additional_kwargs': {}, 'raw': {'answer': "Paul Graham is a well-known British-American programmer, writer, and venture capitalist. He's best known for co-founding the online startup incubator Y Combinator in 2005, which has invested in many successful companies like Dropbox, Airbnb, and Reddit.", 'confidence': 1.0, 'explanation': 'Paul Graham was born in London in 1964 but mo

In [51]:
text = dict_resp["text"]
# fom string to dict
text_dict = json.loads(text)

In [52]:
# create a new Document
doc = Document()
# add a title
doc.add_heading('LLM structured Response', level=1)
# add a paragraph with the response
doc.add_heading('Answer', level=2)
doc.add_paragraph(f"{text_dict['answer']}")
doc.add_heading('Confidence', level=2)
doc.add_paragraph(f"{text_dict['confidence']}")
doc.add_heading('Explanation', level=2)
doc.add_paragraph(f"{text_dict['explanation']}")
doc.add_heading('LLM Name', level=2)
doc.add_paragraph(f"{text_dict['llm_name']}")
# save the document overwrite
doc.save('llm_response.docx')